In [1]:
import os
import shutil
import glob
import pdfplumber
import pandas as pd
import smtplib
import keyring

from email import encoders
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email.mime.image import MIMEImage


In [2]:
#Ler e Ajustar nomes dos PDF Originais
path = './Originais'
for file in glob.iglob(path + '\*.pdf'):
    pdf = pdfplumber.open(file)
    pdf_text = pdf.pages[0].extract_text()
    pdf.close()

    tpDoc = pdf_text[0:15].split()[0]
    
    if tpDoc == 'Documento': #DAS
        pos = pdf_text.find('CNPJ Razão Social')
        cnpj = pdf_text[pos: pos+36].split()[-1]
        cnpj = cnpj.replace('/','').replace('-','').replace('.','').zfill(14)
        newName = 'DAS ' + cnpj + '.pdf'   

    if tpDoc == 'Extrato': #PGDAS-R Extrato
        for pg in pdf.pages:
            pdf_text = pg.extract_text()
            pos = pdf_text.find('CNPJ Estabelecimento: ')
            if pos > -1:
                cnpj = pdf_text[pos: pos+40].split(': ')[1]
                cnpj = cnpj.replace('/','').replace('-','').replace('.','').zfill(14)
                newName = 'PGDAS ' + cnpj + '.pdf'

    if tpDoc == 'Programa': #PGDAS-R Declaratorio
        pos = pdf_text.find('CNPJ Matriz:')
        cnpj = pdf_text[pos: pos+31].split()[-1]
        cnpj = cnpj.replace('/','').replace('-','').replace('.','').zfill(14)
        newName = 'PGDAS ' + cnpj + '.pdf'   

    # print('Renomeando: {} para: {}' .format(file, os.path.join(path , newName)))
    newfile = os.path.join(path , newName)
    if os.path.isfile(newfile):
        print('{} já existe!' .format(newName))
    else:
        os.rename(file, newfile)   

In [3]:
def EnviarEmail(cnpj, empresa, para):
    try:
        msgRoot = MIMEMultipart('related')

        msgRoot['Subject'] = 'DAS (SIMPLES NACIONAL) {} - {}' .format(nmApuracao, empresa)
        msgRoot['From'] = 'Fiscal@unicacontabil.com' 
        msgRoot['To'] = para
        # msgRoot['Cc'] = 'Evy.lima@hotmail.com'
        msgRoot.preamble = 'Multi-part message in MIME format.'

        msgAlternative = MIMEMultipart('alternative')
        msgRoot.attach(msgAlternative)

        msgText = MIMEText('Alternative plain text message.')
        msgAlternative.attach(msgText)

        bodyHtml = '''
            <br>
            Boa tarde,
            <br><br>
            Segue anexo PGDAS / DAS referente a <b>{}</b>.
            <br><br><br>
            <i>Atenciosamente</i>
            <br>
            <img src="cid:image1" alt="Ass" height=140 width=420>        
        ''' .format(nmApuracao)
        
        msgText = MIMEText(bodyHtml, 'html')
        msgAlternative.attach(msgText)

        #Attach Image 
        fp = open('../Assinatura Luan.jpg', 'rb') #Read image 
        msgImage = MIMEImage(fp.read())
        fp.close()

        # Define the image's ID as referenced above
        msgImage.add_header('Content-ID', '<image1>')
        msgRoot.attach(msgImage)
        
        qtAnexo = 0
        #Anexar DAS
        file = './Originais/DAS {}.pdf'.format(cnpj)
        if os.path.isfile(file):
            attachment = open(file,'rb')
            part = MIMEBase('application', 'octet-stream')
            part.set_payload((attachment).read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', "attachment; filename= DAS.pdf")
            msgRoot.attach(part)
            attachment.close()
            qtAnexo = qtAnexo + 1
        else:
            arquivo.write('{} não possui DAS \n'.format(empresa))
            print('{} não possui DAS'.format(empresa)) 
            
        #Anexar PGDAS
        file = './Originais/PGDAS {}.pdf'.format(cnpj)
        if os.path.isfile(file):
            attachment = open(file,'rb')
            part = MIMEBase('application', 'octet-stream')
            part.set_payload((attachment).read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', "attachment; filename=PGDAS-D.pdf")
            msgRoot.attach(part)
            attachment.close()
            qtAnexo = qtAnexo + 1
        
        else:
            arquivo.write('{} não possui PGDAS \n'.format(empresa))
            print('{} não possui PGDAS'.format(empresa)) 
        
        if (qtAnexo > 0):
            #Enviar Email
            server = smtplib.SMTP('smtp.gmail.com: 587')
            server.starttls()
            server.login('Fiscal@unicacontabil.com', keyring.get_password('.py', 'Fiscal'))
            # text = msg.as_string()
            # server.sendmail(msg['From'], para.split(';'), text)
            server.sendmail(msgRoot['From'], para.split(';'), msgRoot.as_string())
            server.quit()
            arquivo.write('{} Ok! \n'.format(empresa))
            print('{} Ok!'.format(empresa))
        else:
            arquivo.write('{} não possui nenhum arquivo para anexar! \n'.format(empresa))
            print('{} não possui nenhum arquivo para anexar!'.format(empresa))
                  
    except:
        arquivo.write('Erro ao enviar para {} \n'.format(empresa))
        print("Erro ao enviar para {}".format(empresa))

def MoverArquivos(cnpj, pasta):
    file = './Originais/DAS {}.pdf'.format(cnpj)
    if os.path.isfile(file):
        shutil.move(file, os.path.join(pasta, 'DAS.pdf'))

    file = './Originais/PGDAS {}.pdf'.format(cnpj)
    if os.path.isfile(file):
        shutil.move(file, os.path.join(pasta, 'PGDAS.pdf'))        

In [4]:
# Carregar Dados Planilha
arq = 'ENVIAR SIMPLES NACIONAL.xlsx'

df = pd.read_excel(arq, header=0)

# filtro = [3011114000125, 8301978000103, 35501843000156, 43664673000159, 37853161000138]
# df2 = df[df['CNPJ'].isin(filtro)]

# df2 = df.head(10)
df2 = df

df2.set_index('CÓDIGO', inplace=True)
df2.fillna('-', inplace=True)
df2


EMPRESA            CNPJ  \
CÓDIGO                                                                 
6                                   ACF MARISTA LTDA   2816376000102   
10                  AFC REABILITACAO ORAL SS LTDA ME   3680000000178   
13                        AGENCIA FRANQUEADA 84 LTDA  37368198000170   
574          ANDREA TRINDADE FERNANDES CORREA EIRELI  33173354000197   
28                APARATTO CORRETORA DE SEGUROS LTDA  21577406000101   
...                                              ...             ...   
1197                CONFIANCA SOLUCOES EIRELI FILIAL  19108740000255   
1234    ENVOLVE MARKETING PROMOCIONAL E EVENTOS LTDA  18019790000112   
1233                      THIAGO RYUJI IWATA EVENTOS  31284142000151   
1232                 LIVE MARKETING E EVENTOS EIRELI  24122799000157   
1205                    ARETHA BRUNA GONZAGA E SILVA  40957644000197   

       INSCRIÇÃO MUNICIPAL                                   EMAIL  \
CÓDIGO                                                               
6                  1461168                   acfmarista@uol.com.br   
10                 1592416                     adrfcosta@gmail.com   
13                  879355                     agencia84@gmail.com   
574                4795474           draandreatfernandes@gmail.com   
28                 3920798             vilelaadvogados@hotmail.com   
...                    ...                                     ...   
1197                     -  financeiro01@solucaoempresarial.net.br   
1234               3503402                    iwata@envolve.com.br   
1233               4660072                    iwata@envolve.com.br   
1232               4175360                    iwata@envolve.com.br   
1205            SEM ACESSO              aretha_bruna06@hotmail.com   

                            EMAIL.1 EMAIL.2 EMAIL.3 EMAIL.4 EMAIL.5 EMAIL.6  \
CÓDIGO                                                                        
6        rogeriorosacosta@gmail.com       -       -       -       -       -   
10                                -       -       -       -       -       -   
13                                -       -       -       -       -       -   
574                               -       -       -       -       -       -   
28                                -       -       -       -       -       -   
...                             ...     ...     ...     ...     ...     ...   
1197                              -       -       -       -       -       -   
1234    atendimento2@envolve.com.br       -       -       -       -       -   
1233    atendimento2@envolve.com.br       -       -       -       -       -   
1232    atendimento2@envolve.com.br       -       -       -       -       -   
1205                              -       -       -       -       -       -   

       C/C  
CÓDIGO      
6        -  
10       -  
13       -  
574      -  
28       -  
...     ..  
1197     -  
1234     -  
1233     -  
1232     -  
1205     -  

[366 rows x 11 columns]

In [5]:
pathDe = './Originais'
pathPara = './Arquivamento'

apuracao = '2022_07'
nmApuracao = 'Julho/2022'

arquivo = open('log.txt','w')

for index, row in df2.iterrows(): 
    # Cria a Pasta da Empresa
    client = str(index) + ' - ' + str(row['EMPRESA']).replace('/', ' ')
    pasta = os.path.join(pathPara, client)
    if not os.path.isdir(pasta): 
        os.mkdir(pasta) 
    
    # Cria a Pasta da Data de Apuração
    pasta = os.path.join(pathPara, client, apuracao)
    if not os.path.isdir(pasta): 
        os.mkdir(pasta) 

    # Lista de Email
    lEmail =[]
    lEmail.append(row['EMAIL'])
    lEmail.append(row['EMAIL.1'])
    lEmail.append(row['EMAIL.2'])
    lEmail.append(row['EMAIL.3'])
    lEmail.append(row['EMAIL.4'])
    lEmail.append(row['EMAIL.5'])
    lEmail.append(row['EMAIL.6'])

    cnpj = str(row['CNPJ']).zfill(14)

    lista = list(filter(lambda x: x != '-', lEmail))
    para = ';'.join(lista)

    #Enviar Email
    EnviarEmail(cnpj, row['EMPRESA'], para)
    
    #Mover Arquivo
    MoverArquivos(cnpj, pasta)
    
arquivo.close()

ACF MARISTA LTDA não possui DAS
ACF MARISTA LTDA não possui PGDAS
ACF MARISTA LTDA não possui nenhum arquivo para anexar!
AFC REABILITACAO ORAL SS LTDA ME não possui DAS
AFC REABILITACAO ORAL SS LTDA ME não possui PGDAS
AFC REABILITACAO ORAL SS LTDA ME não possui nenhum arquivo para anexar!
AGENCIA FRANQUEADA 84 LTDA não possui DAS
AGENCIA FRANQUEADA 84 LTDA não possui PGDAS
AGENCIA FRANQUEADA 84 LTDA não possui nenhum arquivo para anexar!
ANDREA TRINDADE FERNANDES CORREA EIRELI não possui DAS
ANDREA TRINDADE FERNANDES CORREA EIRELI Ok!
APARATTO CORRETORA DE SEGUROS LTDA não possui DAS
APARATTO CORRETORA DE SEGUROS LTDA Ok!
ARIMATEA E ARIMETEA SERVICOS MEDICOS SS LTDA não possui DAS
ARIMATEA E ARIMETEA SERVICOS MEDICOS SS LTDA não possui PGDAS
ARIMATEA E ARIMETEA SERVICOS MEDICOS SS LTDA não possui nenhum arquivo para anexar!
B AMANCIO E COSTA ODONTOLOGIA LTDA não possui DAS
B AMANCIO E COSTA ODONTOLOGIA LTDA não possui PGDAS
B AMANCIO E COSTA ODONTOLOGIA LTDA não possui nenhum arquivo